Nog een keer hetzelfde programma alleen deze keer maak ik het met CustomTkinter. Dat is een Python UI library based on Tkinter die een nieuw, modernere widgets heeft.

In [316]:
# import sys
# !{sys.executable} -m pip install customtkinter

In [317]:
import tkinter
import customtkinter
import numpy as num
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

In [334]:
import sys
import tkinter
import customtkinter

class App():
    def __init__(self):
        customtkinter.set_appearance_mode("dark")  # Modes: "System" (standard), "Dark", "Light"
        customtkinter.set_default_color_theme("blue")  # Themes: "blue" (standard), "green", "dark-blue"

        self.app = customtkinter.CTk()
        self.app.geometry("400x400")
        self.app.title("DataVisualisation App voor Biker")
        
        self.loginAttempts = 0
        self.createLogin()
        
        # wanneer de windows word gesloten sluit hij ook het programma.
        self.app.protocol("WM_DELETE_WINDOW", self.closeApp)
        

    def loginCheck(self):
        # gelezen logingegevens... Super Secure!!!
        creds = pd.read_csv('logingegevens.csv')

        # omzetten naar 2D array. [[username, password], [username, password], enz..]
        credsArr = creds.to_numpy()

        # check login input
        self.username = self.usernameEntry.get()
        self.password = self.passwordEntry.get()

        # loop series van namen en ww
        for arr in credsArr:
            # subset van naam en ww
            if arr[0] == self.username:
                if arr[1] == self.password:
                    # username en password zijn correct
                    self.loginOut(True)
                    return
        
        self.loginOut(False)
                
    def loginOut(self, out):
        if out == True:
            m = Main()
            # m.main.mainloop()
            self.closeApp()

        # login pogingen staat op 2. de user heeft namelijk maar 3 pogingen
        if self.loginAttempts >= 2:
            # todo open window met message na app.destroy()
            e = ErrorWindow(reden="WW")
            e.window.mainloop()
            self.closeApp()
            
            
        elif out == False:
            self.loginAttempts = self.loginAttempts + 1
            label = customtkinter.CTkLabel(master=self.loginWindow, text=f'Logingegevens zijn incorrect.')
            label.pack()

            labelMessage = customtkinter.CTkLabel(master=self.loginWindow, text=f'Dit is poging {self.loginAttempts}. u heeft nog {3- self.loginAttempts} pogingen')
            labelMessage.pack()
    
    def closeApp(self):
        self.app.quit()
        self.app.destroy()

    def startApp(self):
        self.app.mainloop()


    def createLogin(self):
        self.loginWindow = customtkinter.CTkFrame(master=self.app)
        self.loginWindow.pack(pady=20, padx=60, fill="both", expand=True)

        self.usernameEntry = customtkinter.CTkEntry(master=self.loginWindow, placeholder_text="Please enter your username:")
        self.usernameEntry.pack(pady=10, padx=10)

        self.passwordEntry = customtkinter.CTkEntry(master=self.loginWindow, placeholder_text="Please enter your password", show="*")
        self.passwordEntry.pack()

        self.button_1 = customtkinter.CTkButton(master=self.loginWindow, text="Login!", command=self.loginCheck)
        self.button_1.pack(pady=10, padx=10)
        
        self.close = customtkinter.CTkButton(master=self.loginWindow, text="Afsluiten", command=self.closeApp)
        self.close.pack()



In [335]:
class Main():
    def __init__(self):
        main = customtkinter.CTk()
        main.geometry("600X800")
        # soort van grid layout om een sidebar en resultaten ding te maken
        main.columnconfigure(0, weight=1)
        main.columnconfigure(1, weight=3)
        self.createFrames(main)

        main.mainloop()
        # wanneer de windows word gesloten sluit hij ook het programma.
        self.main.protocol("WM_DELETE_WINDOW", self.closeApp)


        

    def createFrames(self, main):
        self.frameSide = customtkinter.CTkFrame(master=main)
        self.frameSide.grid(column=0, row=0, sticky=customtkinter.W)
        self.fillLeftFrame()

        self.frameMain = customtkinter.CTkFrame(master=main)
        self.frameMain.grid(column=1, row=0, sticky=customtkinter.E)
        self.fillRightFrame()

    def fillLeftFrame(self):
        self.label = customtkinter.CTkLabel(master=self.frameSide, text="Kies hieronder data dat je wilt zien")
        self.label.pack(pady=10, padx=10)

    def fillRightFrame(self):
        self.label = customtkinter.CTkLabel(master=self.frameMain, text="Right side")
        self.label.pack()

    def closeApp(self):
        self.main.quit()
        self.main.destroy()


In [338]:
class ErrorWindow():
    def __init__(self, reden):
        self.reden = reden
        self.window = customtkinter.CTk()

        if self.reden == "WW":
            self.message = "U heeft 3 keer de verkeerde logingegevens ingevuld. De applicatie sluit nu."
        else:
            self.message = "Error is unknown"            

        errorMessage = customtkinter.CTkLabel(master=self.window, text=self.message)
        errorMessage.pack()

        # wanneer de windows word gesloten sluit hij ook het programma.
        self.window.protocol("WM_DELETE_WINDOW", self.closeApp)


    def closeApp(self):
        self.window.quit()
        self.window.destroy()


In [339]:
x = App()
x.startApp()


Er is een soort van bug met CTK dat het programma blijft runnen wanneer de app is gesloten. Dit komt omdat de window van zichzelf op state: withdrawn springt zodra CTK met eigen aanpassingen een TK window aanmaakt. Hier is nog geen oplossing voor... dus het programma moet handmatig gesloten worden door op het stopteken te klikken

APP SLUIT NIET CORRECT ALLEEN GOED BIJ ERRORMESSAGE
